# 03 · Backtest & Visualization

Validate the SPX-focused ML signals using a vectorized long/short simulator and visualize equity curves.


**Goals**
- Load cached features, prices, and trained model
- Generate probability-driven signals and flip into positions
- Run a fast vectorized backtest with transaction costs
- Visualize performance metrics and sanity-check exposures


In [ ]:
from pathlib import Path

import joblib
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

from momentum_lib import (
    bootstrap_env,
    generate_signals,
    backtest_signals,
)

sns.set_theme(style="whitegrid")
bootstrap_env(Path("../.env"))
data_dir = Path("../data")
features = pd.read_csv(data_dir / "features.csv", index_col=0)
prices = pd.read_csv(data_dir / "prices.csv", parse_dates=[0], index_col=0)
model = joblib.load(data_dir / "uprx_model.joblib")

overlap = features.index.intersection(prices.index)
X = features.loc[overlap]
prices = prices.loc[overlap]
print(f"Aligned frame shape: {X.shape}")


In [ ]:
signals = generate_signals(model, X, threshold=0.55)
results = backtest_signals(prices, signals)
results.head()


In [ ]:
plt.figure(figsize=(8, 4))
sns.histplot(results["strategy_return"], bins=50, kde=True)
plt.title("Strategy return distribution")
plt.xlabel("Return per bar")
plt.show()



In [ ]:
results.attrs["stats"]


In [ ]:
signal_frame = pd.DataFrame({"signal": signals, "ret": results["strategy_return"]})
plt.figure(figsize=(6, 4))
sns.boxplot(data=signal_frame, x="signal", y="ret")
plt.title("Returns conditioned on signal direction")
plt.show()



In [ ]:
fig = px.line(results.reset_index(), x="index", y="equity", title="Strategy Equity Curve")
fig.show()


In [ ]:
signals.value_counts(normalize=True).rename("signal_share")
